In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3,0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2,0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1,0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1,0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0,0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0,0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0,2,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3,0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0,0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset=dataset.astype(int) 

In [5]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [6]:
dataset['classification_yes'].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [7]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]


In [8]:
dependent=dataset[['classification_yes']]

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [11]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

param_grid={'n_neighbors':[5,7],'metric':['minkowski','collable'],'p':[1,2],'weights':['uniform','distance']}

grid=GridSearchCV(KNeighborsClassifier(),param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\ProgramData\anaconda3\Lib\site-packages\

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['minkowski', 'collable'],
                         'n_neighbors': [5, 7], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [12]:
re=grid.cv_results_

In [13]:
y_pred=grid.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)

In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for best parameter{}".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter{'metric': 'minkowski', 'n_neighbors': 5, 'p': 1, 'weights': 'uniform'} 0.9626932787797391


In [16]:
print(cm)

[[51  0]
 [ 5 77]]


In [17]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.91      1.00      0.95        51
           1       1.00      0.94      0.97        82

    accuracy                           0.96       133
   macro avg       0.96      0.97      0.96       133
weighted avg       0.97      0.96      0.96       133



In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9934241989478718

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005598,0.004136,0.118674,0.047593,minkowski,5,1,uniform,"{'metric': 'minkowski', 'n_neighbors': 5, 'p':...",0.926978,0.981233,0.944023,0.981217,0.981217,0.962934,0.023038,1
1,0.003944,0.006051,0.037694,0.008723,minkowski,5,1,distance,"{'metric': 'minkowski', 'n_neighbors': 5, 'p':...",0.926978,0.981233,0.944023,0.981217,0.981217,0.962934,0.023038,1
2,0.000941,0.001881,0.034919,0.007957,minkowski,5,2,uniform,"{'metric': 'minkowski', 'n_neighbors': 5, 'p':...",0.963284,0.962636,0.925524,0.981217,0.962573,0.959047,0.018213,4
3,0.006322,0.006074,0.038671,0.005673,minkowski,5,2,distance,"{'metric': 'minkowski', 'n_neighbors': 5, 'p':...",0.963284,0.981233,0.925524,0.981217,0.962573,0.962766,0.020341,3
4,0.000000,0.000000,0.031151,0.001891,minkowski,7,1,uniform,"{'metric': 'minkowski', 'n_neighbors': 7, 'p':...",0.908877,0.944161,0.907035,0.981217,0.981217,0.944501,0.032769,7
5,0.010297,0.008626,0.036496,0.007943,minkowski,7,1,distance,"{'metric': 'minkowski', 'n_neighbors': 7, 'p':...",0.908877,0.944161,0.907035,0.981217,0.981217,0.944501,0.032769,7
6,0.003832,0.006068,0.036587,0.004709,minkowski,7,2,uniform,"{'metric': 'minkowski', 'n_neighbors': 7, 'p':...",0.926978,0.962636,0.925524,0.981217,0.962573,0.951786,0.021933,5
7,0.002626,0.003390,0.037062,0.003022,minkowski,7,2,distance,"{'metric': 'minkowski', 'n_neighbors': 7, 'p':...",0.926978,0.962636,0.925524,0.981217,0.962573,0.951786,0.021933,5
8,0.001235,0.002470,0.000000,0.000000,collable,5,1,uniform,"{'metric': 'collable', 'n_neighbors': 5, 'p': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
9,0.001001,0.002002,0.000000,0.000000,collable,5,1,distance,"{'metric': 'collable', 'n_neighbors': 5, 'p': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9


In [21]:
age_input=int(input('Enter age:'))
Bp=float(input("Enter bp:"))
Al_input=int(input("Enter al 0 or 1:"))
Su_input=float(input("Enter su:"))
bgr_input=float(input("Enter bgr:"))
bu_input=float(input("Enter bu:"))
sc_input=float(input("enter sc:"))
sod_input=float(input("enter sod:"))
pot_input=float(input("enter pot:"))
hrmo_input=float(input("enter hrmo:"))
pcv_input=float(input("enter pcv:"))
wc_input=float(input("wc:"))
rc_input=float(input("rc:"))
sg_b_input=float(input("sg_b:"))
sg_c_input=float(input("sg_c:"))
sg_d_input=float(input("sg_d:"))
sg_e_input=float(input("sg_e:"))
rbc_normal_input=int(input("rbc_normal 0 or 1:"))
pc_normal_input=int(input("pc_normal 0 or 1:"))
pcc_present_input=int(input("pcc_present 0 or 1:"))
ba_present_input=int(input("ba_present 0 or 1:"))
htn_yes_input=int(input("htn_yes 0 or 1:"))
dm_yes_input=int(input("dm_yes 0 or 1:"))
cad_yes_input=int(input("cad_yes 0 or 1:"))
appet_yes_input=int(input("appet_yes 0 or 1:"))
pe_yes_input=int(input("pe_yes 0 or 1:"))
ane_yes_input=int(input("ane_yes 0 or 1:"))

Enter age:36
Enter bp:78
Enter al 0 or 1:1
Enter su:3.0
Enter bgr:150
Enter bu:26.30
enter sc:2.50000
enter sod:136
enter pot:4.20
enter hrmo:12.86539
enter pcv:38
wc:9000.236
rc:6.2
sg_b:32
sg_c:34
sg_d:36
sg_e:38
rbc_normal 0 or 1:1
pc_normal 0 or 1:0
pcc_present 0 or 1:1
ba_present 0 or 1:0
htn_yes 0 or 1:1
dm_yes 0 or 1:0
cad_yes 0 or 1:0
appet_yes 0 or 1:1
pe_yes 0 or 1:0
ane_yes 0 or 1:1


In [22]:
Future_Prediction=grid.predict([[age_input,Bp,Al_input,Su_input,bgr_input,bu_input,sc_input,sod_input,pot_input,hrmo_input,pcv_input,wc_input,rc_input,sg_b_input,sg_c_input,sg_d_input,sg_e_input,rbc_normal_input,pc_normal_input,pcc_present_input,ba_present_input,htn_yes_input,dm_yes_input,cad_yes_input,appet_yes_input,pe_yes_input,ane_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
